# Libraries

In [1]:
import pickle as pkl
import pandas as pd

In [2]:
algo_name = "UserKNN"
data_strategies = [
    "uniformly_random",
    "popularity_good",
    "popularity_bad",
    "popularity_good_for_bp_ur",
    "popularity_bad_for_bp_ur",
]

# Analysis

## LKPY

In [3]:
algo_versions = {
    "UserKNN": [
        {"min_nbrs": 1, "min_sim": 0},
        {"min_nbrs": 2, "min_sim": 0},
        {"min_nbrs": 1, "min_sim": -1},
        {"min_nbrs": 2, "min_sim": -1},
    ],
}

In [4]:
versions = algo_versions[algo_name]

In [5]:
file_location = "experimental_results/" + algo_name + "/"

In [6]:
results = []
for data_strategy in data_strategies:
    for args in versions:
        file = open(file_location + data_strategy + "_" + str(args) + ".pkl", "rb")
        result = pkl.load(file)
        results.append(result)

In [7]:
stringed_versions = [str(args) for args in versions]

In [8]:
# Initialize empty lists for the two halves
center = []
min_sim = []

# Split the strings and populate the lists
for s in stringed_versions:
    parts = s.split(", ")

    center.append(parts[0].split(" ")[-1])
    min_sim.append(parts[1].split(" ")[-1].split("}")[0])

In [9]:
over_common = ["False"]

In [10]:
index = pd.MultiIndex.from_product(
    [data_strategies, min_sim, center, over_common],
    names=["DataStrategy", "MinimumSimilarity", "MinimumNeighbours", "OverCommon"],
).drop_duplicates()

In [11]:
results_lkpy = pd.DataFrame(results, index=index)

## Cornac

In [12]:
algo_versions = {
    "CornacUserKNN": [{"center": True}],
}
algo_name = "CornacUserKNN"

In [13]:
versions = algo_versions[algo_name]

In [14]:
file_location = "experimental_results/" + algo_name + "/"

In [15]:
results = []
for data_strategy in data_strategies:
    for args in versions:
        file = open(file_location + data_strategy + "_" + str(args) + ".pkl", "rb")
        result = pkl.load(file)
        results.append(result)

In [16]:
stringed_versions = [str(args) for args in versions]

In [17]:
# Initialize empty lists for the two halves
center = []
min_sim = []
# Split the strings and populate the lists
for s in stringed_versions:
    parts = s.split(": ")
    center.append(parts[-1].split("}")[0])
    min_sim.append("-1")

In [18]:
over_common = ["True"]

In [19]:
index = pd.MultiIndex.from_product(
    [data_strategies, min_sim, center, over_common],
    names=["DataStrategy", "MinimumSimilarity", "MinimumNeighbours", "OverCommon"],
).drop_duplicates()

In [20]:
results_cornac = pd.DataFrame(results, index=index)

In [21]:
results_cornac.index = results_cornac.index.set_levels(
    results_cornac.index.levels[1].str.replace("-1", "1"), level=2
)

# Merge

In [22]:
user_knn_metrics = (
    pd.concat([results_lkpy, results_cornac])
    .reset_index()
    .sort_values(
        ["DataStrategy", "MinimumSimilarity", "OverCommon", "MinimumNeighbours"]
    )
    .set_index(["DataStrategy", "MinimumSimilarity", "OverCommon", "MinimumNeighbours"])
    .reindex(data_strategies, level=0)
)

In [23]:
metrics_order = ["pop_corr", "ARP", "ave_PL", "ACLT", "AggDiv", "RMSE", "NDCG"]
user_knn_metrics = user_knn_metrics[metrics_order]

In [24]:
user_knn_metrics = user_knn_metrics.rename(
    columns={"pop_corr": "PopCorr", "ave_PL": "PL", "ACLT": "APLT", "NDCG": "NDCG@10"}
)

In [25]:
user_knn_metrics = user_knn_metrics.rename(
    index={
        "uniformly_random": "Scenario 1",
        "popularity_good": "Scenario 2",
        "popularity_bad": "Scenario 3",
        "popularity_good_for_bp_ur": "Scenario 4",
        "popularity_bad_for_bp_ur": "Scenario 5",
    }
)

In [26]:
user_knn_metrics = user_knn_metrics.reindex(["1", "2", "5", "10"], level=3)

In [27]:
user_knn_metrics['RealPopCorr'] = user_knn_metrics.PopCorr.apply(lambda x: x[0])
user_knn_metrics['Significance'] = user_knn_metrics.PopCorr.apply(lambda x: True if x[1]<0.005 else False)
user_knn_metrics['PopCorr'] = user_knn_metrics.RealPopCorr 
user_knn_metrics = user_knn_metrics.drop('RealPopCorr', axis=1)

In [30]:
user_knn_metrics

PopCorr  \
DataStrategy MinimumSimilarity OverCommon MinimumNeighbours             
Scenario 1   -1                False      1                  0.018079   
                                          2                  0.418226   
                               True       1                  0.003513   
             0                 False      1                  0.101105   
                                          2                  0.614905   
Scenario 2   -1                False      1                  0.595504   
                                          2                  0.614367   
                               True       1                  0.604307   
             0                 False      1                  0.551960   
                                          2                  0.562015   
Scenario 3   -1                False      1                  0.559173   
                                          2                  0.727548   
                               True       1                  0.521765   
             0                 False      1                  0.024709   
                                          2                  0.160765   
Scenario 4   -1                False      1                  0.253427   
                                          2                  0.772315   
                               True       1                  0.184022   
             0                 False      1                  0.587823   
                                          2                  0.700713   
Scenario 5   -1                False      1                  0.086560   
                                          2                  0.623256   
                               True       1                  0.057401   
             0                 False      1                  0.136474   
                                          2                  0.612128   

                                                                  ARP  \
DataStrategy MinimumSimilarity OverCommon MinimumNeighbours             
Scenario 1   -1                False      1                  0.002059   
                                          2                  0.003528   
                               True       1                  0.001959   
             0                 False      1                  0.002723   
                                          2                  0.005088   
Scenario 2   -1                False      1                  0.020677   
                                          2                  0.022179   
                               True       1                  0.015046   
             0                 False      1                  0.027255   
                                          2                  0.027052   
Scenario 3   -1                False      1                  0.007501   
                                          2                  0.008210   
                               True       1                  0.006448   
             0                 False      1                  0.002130   
                                          2                  0.002811   
Scenario 4   -1                False      1                  0.003283   
                                          2                  0.005797   
                               True       1                  0.002873   
             0                 False      1                  0.007840   
                                          2                  0.013732   
Scenario 5   -1                False      1                  0.002568   
                                          2                  0.004503   
                               True       1                  0.002348   
             0                 False      1                  0.002692   
                                          2                  0.004624   

                                                                     PL  \
DataStrategy MinimumSimilarity OverCommon M

In [28]:
print(user_knn_metrics.drop(['APLT','Significance'], axis=1).round(3).to_latex())

\begin{tabular}{llllrrrrrr}
\toprule
           &   &       &   &  PopCorr &    ARP &       PL &  AggDiv &   RMSE &  NDCG@10 \\
DataStrategy & MinimumSimilarity & OverCommon & MinimumNeighbours &          &        &          &         &        &          \\
\midrule
Scenario 1 & -1 & False & 1 &    0.018 &  0.002 &  -32.285 &   0.400 &  3.502 &    0.001 \\
           &   &       & 2 &    0.418 &  0.004 &   21.252 &   0.681 &  3.352 &    0.003 \\
           &   & True & 1 &    0.004 &  0.002 &  -35.746 &   0.400 &  3.337 &    0.001 \\
           & 0 & False & 1 &    0.101 &  0.003 &  -12.827 &   0.340 &  3.624 &    0.002 \\
           &   &       & 2 &    0.615 &  0.005 &   65.440 &   0.595 &  3.464 &    0.005 \\
Scenario 2 & -1 & False & 1 &    0.596 &  0.021 &  426.621 &   0.197 &  1.188 &    0.019 \\
           &   &       & 2 &    0.614 &  0.022 &  447.618 &   0.339 &  1.190 &    0.021 \\
           &   & True & 1 &    0.604 &  0.015 &  305.197 &   0.238 &  1.150 &    0.013 \\
     

In [29]:
import pickle

with open("metrics_combined/all_user_knn.pkl", "wb") as f:
    pickle.dump(user_knn_metrics.drop("APLT", axis=1).round(3), f)